In [54]:
import os 
import numpy as np
import pandas as pd 
from functools import reduce
from tqdm import tqdm 
import collections

In [9]:
files = os.listdir('/gpfs/data/luilab/CARE_data/')

In [10]:
files

['TBIBT578WHF_ImagingMR_1554229921742_Q29vFfB61FsKwhAF1cZc7nS9R_2.zip',
 'TBICK676PXX_ImagingMR_1553865396843.jpg',
 'TBIDB663GHZ_ImagingMR_1554224892097.jpg',
 'TBIDW824RKY_ImagingDiffusion_1555097469229_7hnCti2JwFjmOudNEFtKNTvqg_16.zip',
 'TBIJD625EHC_ImagingMR_1554209886345.jpg',
 'TBIPU112TXT_ImagingDiffusion_1555351186150_ZSI6oYbsCqzsvrSPVFyw9yulL_9.zip',
 'TBIXV229WNQ_ImagingDiffusion_1555095409832.jpg',
 'TBIPN535ZAR_ImagingMR_1554211196510_42jduIwuDvpxHMVSapEXNIs7r_7.zip',
 'TBIFP042ZF9_ImagingDiffusion_1555354848508_ipuKhi9G20D9VUSuqpCRghg80_8.zip',
 'TBIZN166LJY_ImagingFunctionalMR_1554323830223.jpg',
 'TBIRV085JAP_ImagingMR_1553870679950_YcvT5AFkEWu0g89LTTCOsGjP3_5.zip',
 'TBIBU887ZBJ_ImagingMR_1553259694849.jpg',
 'TBITM629CB6_ImagingMR_1553795033102.jpg',
 'TBIMY671HC9_ImagingDiffusion_1555354614292_1PIkbBdjGoX9tsF6CVrRsorjc_8.zip',
 'TBIEX362VH3_ImagingDiffusion_1555096709513.jpg',
 'TBIKM128BKE_ImagingMR_1553863924065_om3lwrWi5sAysZL6UHgvdPyie_4.zip',
 'TBIHA813KG4_Imagi

Look deeper into MR Imaging 

In [40]:
# get all mr images file 
mrimage_zips = [file for file in files if 'ImagingMR' in file and '.zip' in file]
print('imagingMR zip file number', len(mrimage_zips))
# copy them to separate folder: 
'''
for mrimage_zip in tqdm(mrimage_zips):
    sour_file = os.path.join('/gpfs/data/luilab/CARE_data/', mrimage_zip)
    dest_dir = '/gpfs/data/luilab/CARE_MRImaging/'
    os.system(f'cp {sour_file} {dest_dir}')
'''
# count the number of zip copied:
print('copied files:', len([x for x in os.listdir('/gpfs/data/luilab/CARE_MRImaging/') if 'zip' in x]))
'''
#unzip zip files:
for mrimage_zip in tqdm(mrimage_zips):
    zip_file = os.path.join('/gpfs/data/luilab/CARE_MRImaging/', mrimage_zip)
    dest = zip_file[:-4]
    os.system('unzip {} -d {}'.format(zip_file, dest))
'''
# construct sequence record 
dirs = [os.path.join('/gpfs/data/luilab/CARE_MRImaging/', file) for file in os.listdir('/gpfs/data/luilab/CARE_MRImaging/')]
dirs = [x for x in dirs if os.path.isdir(x)]
seqs = []
for x in tqdm(dirs):
    seqs.append((x.split('/')[-1], list(os.walk(x))[2][1]))
    
    


imagingMR zip file number 5521
copied files: 5521


In [46]:
# count sequence 
counts = list(zip(*np.unique([x[1][0] for x in seqs], return_counts=True)))
counts = sorted(counts, key=lambda x:-x[1])

In [49]:
seqs_dict = {x[0]:x[1][0] for x in seqs}

In [64]:
# read MR imaging file 
TI_file = '/gpfs/data/luilab/FITBIR_META_DATA/assessments/query_result_ImagingMR_2019-07-17T15-31-508831927565068657063.csv'
T1_info = pd.read_csv(TI_file, low_memory=False)
tps = ['24-48 hours', 'Asymptomatic', '7 days Post-Unrestricted Return to Play', '6 months post-injury']
T1_seq = collections.defaultdict(list)
missing_f = []
for i, row in T1_info.iterrows():
    guid = row['ImagingMR.Main.GUID']
    subjectid = str((row['ImagingMR.Main.SubjectIDNum']))
    tpid = str(int(tps.index(row['ImagingMR.Main.GeneralNotesTxt'])))
    key = guid+'-'+subjectid+'-'+tpid
    f = row['ImagingMR.Image Information.ImgFile'].split('/')[-1][:-4]
    if f in seqs_dict:
        T1_seq[key].append(seqs_dict[f])
    else:
        missing_f.append(f)


In [72]:
print('mri not copies from labdrive by research administrator:', len(missing_f))
print(list(zip(*np.unique([x[-1] for x in missing_f], return_counts=True))))

mri not copies from labdrive by research administrator: 306
[('3', 281), ('4', 18), ('5', 4), ('6', 1), ('8', 1), ('9', 1)]


In [76]:
avail_dti = pd.read_csv('./dti_info.csv')
avail_dti['key'] = avail_dti['ImagingDiffusion.Main.GUID']  \
                   + '-' + avail_dti['ImagingDiffusion.Main.SubjectIDNum'].apply(lambda x: str(int(x))) \
                   + '-' + avail_dti['ImagingDiffusion.Main.GeneralNotesTxt'].apply(lambda x : str(tps.index(x)))
dti_keys = set(avail_dti['key'].values)

In [78]:
dti_ti_seq = {key:T1_seq[key] for key in dti_keys}

In [80]:
ti_seq_dti = collections.defaultdict(list)
for key in dti_ti_seq:
    sequences = dti_ti_seq[key]
    for sequence in sequences:
        seq = sequence.split('-')[-1]
        ti_seq_dti[seq].append(key)

In [83]:
# calculate WSM distribution 
case_ids = set(avail_dti[avail_dti['cohort'] == 0]['ImagingDiffusion.Main.GUID'].values)
control1_ids =  set(avail_dti[avail_dti['cohort'] == 1]['ImagingDiffusion.Main.GUID'].values)
control2_ids = set(avail_dti[avail_dti['cohort'] == 2]['ImagingDiffusion.Main.GUID'].values)

In [87]:
len(dti_keys)

478

In [86]:
for seq_key in ti_seq_dti:
    print(seq_key)
    files = set(ti_seq_dti[seq_key])
    print(len(files))
    key_seqs_case = [x for x in files if x.split('-')[0] in case_ids]
    key_seqs_control1 = [x for x in files if x.split('-')[0] in control1_ids]
    key_seqs_control2 = [x for x in files if x.split('-')[0] in control2_ids]
    print("cases: ", len(key_seqs_case))
    print(list(zip(*np.unique([x.split('-')[-1] for x in key_seqs_case], return_counts=True))))
    print("control1: ", len(key_seqs_control1))
    print(list(zip(*np.unique([x.split('-')[-1] for x in key_seqs_control1], return_counts=True))))
    print("control2: ", len(key_seqs_control2))
    print(list(zip(*np.unique([x.split('-')[-1] for x in key_seqs_control2], return_counts=True))))
    

ep2d_Axial_pasl
276
cases:  85
[('0', 21), ('1', 29), ('2', 20), ('3', 15)]
control1:  85
[('0', 23), ('1', 23), ('2', 21), ('3', 18)]
control2:  106
[('0', 30), ('1', 28), ('2', 31), ('3', 17)]
Axial_3D_PASL
81
cases:  29
[('0', 8), ('1', 10), ('2', 5), ('3', 6)]
control1:  24
[('0', 7), ('1', 6), ('2', 6), ('3', 5)]
control2:  28
[('0', 7), ('1', 7), ('2', 8), ('3', 6)]
ep2d_tra_pasl
92
cases:  35
[('0', 9), ('1', 11), ('2', 8), ('3', 7)]
control1:  24
[('0', 7), ('1', 6), ('2', 6), ('3', 5)]
control2:  33
[('0', 9), ('1', 9), ('2', 9), ('3', 6)]
gre_field_mapping
360
cases:  115
[('0', 32), ('1', 35), ('2', 25), ('3', 23)]
control1:  109
[('0', 29), ('1', 29), ('2', 27), ('3', 24)]
control2:  136
[('0', 37), ('1', 36), ('2', 39), ('3', 24)]
MPRAGE
360
cases:  115
[('0', 32), ('1', 35), ('2', 25), ('3', 23)]
control1:  109
[('0', 29), ('1', 29), ('2', 27), ('3', 24)]
control2:  136
[('0', 37), ('1', 36), ('2', 39), ('3', 24)]
T2starW
269
cases:  82
[('0', 22), ('1', 26), ('2', 18), (

In [88]:
mcw_keys = 'TBICR864CMD-2515-2', 'TBIVJ364XAG-1919-2', 'TBIUV221NTZ-1732-3', 'TBIUJ419GBT-2387-1', 'TBIEP779ECB-1318-0', 'TBIYH619YFD-1320-1', 'TBIKM128BKE-1381-3', 'TBIVX914PTT-2101-2', 'TBICJ936JMA-821-1', 'TBIEU157ZJ4-2878-2', 'TBIEF797KNL-1772-1', 'TBIWK874BJD-1700-1', 'TBIBY712FUD-819-2', 'TBITC277FAU-2481-0', 'TBIMK372UE8-684-2', 'TBIKX518XMT-1789-3', 'TBIBU887ZBJ-876-0', 'TBIXB731TKC-2541-2', 'TBIBL649MKK-1163-0', 'TBIVH382BZ8-913-3', 'TBICZ031GEX-989-2', 'TBIKY573LNX-1316-1', 'TBIUZ494TV7-1377-3', 'TBIVX914PTT-657-2', 'TBITC277FAU-2106-2', 'TBIYY025VHJ-1394-1', 'TBIRK940MKM-604-3', 'TBILW475GX0-2889-3', 'TBITG047JT0-1754-3', 'TBICJ342FKR-1380-2', 'TBIUW803JYZ-741-0', 'TBIJA036XXJ-820-3', 'TBIEU157ZJ4-2878-0', 'TBIUW803JYZ-741-3', 'TBIWK874BJD-1700-2', 'TBITU028JF7-1084-1', 'TBIKX518XMT-1789-1', 'TBIKE672MCW-2144-1', 'TBIVJ364XAG-1919-1', 'TBICR864CMD-2515-0', 'TBICZ031GEX-989-3', 'TBIBL649MKK-1163-2', 'TBIUV221NTZ-1732-1', 'TBIKM128BKE-1381-2', 'TBIKY573LNX-1316-3', 'TBIVX914PTT-2101-0', 'TBICJ936JMA-821-0', 'TBITC277FAU-2841-1', 'TBIYY025VHJ-1394-3', 'TBILW475GX0-2889-0', 'TBITG047JT0-1754-2', 'TBIUH587VZY-1382-1', 'TBICJ936JMA-821-2', 'TBIEP779ECB-1318-2', 'TBIJA036XXJ-820-1', 'TBIBU887ZBJ-876-3', 'TBIXB731TKC-2541-0', 'TBIVH382BZ8-913-1', 'TBITC277FAU-2481-2', 'TBIUZ494TV7-1377-1', 'TBICJ342FKR-1380-3', 'TBIUW803JYZ-741-2', 'TBIWK874BJD-1700-0', 'TBITU028JF7-1084-0', 'TBIKX518XMT-1789-0', 'TBILW475GX0-2889-2', 'TBIBU887ZBJ-876-1', 'TBIXB731TKC-2541-3', 'TBICZ031GEX-989-1', 'TBIBL649MKK-1163-1', 'TBIUV221NTZ-1732-0', 'TBIKM128BKE-1381-0', 'TBIKY573LNX-1316-2', 'TBIVX914PTT-657-3', 'TBIMK372UE8-684-3', 'TBITC277FAU-2841-0', 'TBIYY025VHJ-1394-2', 'TBIVJ364XAG-1919-3', 'TBITG047JT0-1754-0', 'TBIUH587VZY-1382-0', 'TBIVX914PTT-2101-3', 'TBICJ936JMA-821-3', 'TBIEP779ECB-1318-1', 'TBIJA036XXJ-820-0', 'TBIEU157ZJ4-3191-0', 'TBIEF797KNL-1772-3', 'TBIVH382BZ8-913-0', 'TBIBY712FUD-819-0', 'TBITC277FAU-2481-1', 'TBIUZ494TV7-1377-0', 'TBIVJ364XAG-1919-0', 'TBICR864CMD-2515-1', 'TBIBL649MKK-1163-3', 'TBIUV221NTZ-1732-2', 'TBIUJ419GBT-2387-0', 'TBIYY025VHJ-1394-0', 'TBIKM128BKE-1381-1', 'TBIVX914PTT-2101-1', 'TBIEU157ZJ4-2878-1', 'TBIEF797KNL-1772-0', 'TBIWK874BJD-1700-3', 'TBITU028JF7-1084-2', 'TBIBY712FUD-819-1', 'TBITC277FAU-2106-0', 'TBIMK372UE8-684-0', 'TBIKX518XMT-1789-2', 'TBICJ342FKR-1380-1', 'TBIEP779ECB-1318-3', 'TBIJA036XXJ-820-2', 'TBIRK940MKM-604-2', 'TBILW475GX0-2889-1', 'TBITG047JT0-1754-1', 'TBIBU887ZBJ-876-2', 'TBIXB731TKC-2541-1', 'TBIVH382BZ8-913-2', 'TBIUZ494TV7-1377-2', 'TBIKY573LNX-1316-0', 'TBITC277FAU-2106-1'

In [93]:
# 
print('any MCW data has MPRAGE', np.any([x in set(ti_seq_dti['MPRAGE']) for x in mcw_keys]))

any MCW data has MPRAGE False


Check overlap with functional MRI 

In [108]:
files = os.listdir('/gpfs/data/luilab/CARE_data/')
# get all mr images file 
fmrimage_zips = [file for file in files if 'ImagingFunctionalMR' in file]
print('imagingMR zip file number', len(fmrimage_zips))
# copy them to separate folder: 

imagingMR zip file number 798


In [110]:
# total on labdrive 17842
print('research administrator failed to move:', 17842 - len(files))

research administrator failed to move: 1087


Read csv files

In [3]:
SAVEPATH = '../assessments/'

In [4]:
PATH = '/gpfs/data/luilab/CARE_data/'
files = [os.path.join(PATH, x) for x in os.listdir(PATH)]
files = sorted(files)
print(len(files))

16755


In [ ]:
TBIMU702RAJ_ImagingDiffusion_1569269142360_sAxodcuzMiYrHThPvLSlEE8CY_8

In [6]:
[x for x in files if 'TBIMU702RAJ_ImagingDiffusion_' in x and 'sAxodcuzMiYrHThPvLSlEE8CY_8' in x]

['/gpfs/data/luilab/CARE_data/TBIMU702RAJ_ImagingDiffusion_1555360116478_sAxodcuzMiYrHThPvLSlEE8CY_8.zip']

Save all not None 

In [5]:
dfs = []
for file in files:
    data = pd.read_csv(file, low_memory=False)
    data = data.dropna(axis=1, how='all')
    data = data.loc[:, ~data.columns.str.contains('ID|Administration|Main|Data')]
    max_num = data.max()
    min_num = data.min()
    d = {'min_value':min_num, 'max_value':max_num}
    df = pd.DataFrame(data=d)
    dfs.append(df)
    
df_max_min = pd.concat(dfs)
#df_max_min.to_csv(os.path.join(SAVEPATH, 'score_max_min.csv'))

Count number of GUID in each assessment 

In [6]:
files_appendix = [x for x in files if 'Appdx' in x]

def get_GUID_num(file):
    data = pd.read_csv(file, low_memory=False)
    GUID_key = [x for x in data.keys() if 'GUID' in x]
    num_GUID = len(set(data[GUID_key[0]]))
    return num_GUID

names = [ x[:x.find('Appdx')].split('/')[-1] for x in files_appendix]
num_GUIDs = [get_GUID_num(file) for file in files_appendix]

d = dict(assessment_name=names, number_unique_GUID=num_GUIDs)
df_unique_GUID = pd.DataFrame(data=d)

#df_unique_GUID.to_csv(os.path.join(SAVEPATH, 'assessment_unique_GUID.csv'))

In [7]:
df_unique_GUID

,assessment_name,number_unique_GUID
0,query_result_ANAM,812
1,query_result_AXON,1581
2,query_result_BESS_,33519
3,query_result_BSI18_,32892
4,query_result_BSSS_,29997
5,query_result_BVMTR,117
6,query_result_CNSDomainSco_,1780
7,query_result_CRT,2999
8,query_result_DANA,100
9,query_result_DemogrFITBIR_,34561


Imaging Meta Data

In [22]:
diffusion_imaging_meta = os.path.join(PATH, 'query_result_ImagingDiffusion_2019-07-17T15-35-37668336643334196993.csv')
df = pd.read_csv(diffusion_imaging_meta)
GUIDs_in_diffusion_imaging_csv = set(df['ImagingDiffusion.Main.GUID'].values)
print(len(GUIDs_in_diffusion_imaging_csv))

312


Data File

Compared updated DTI and data folder

In [54]:
# get a dictionary of all DiffusionImaging files we have 
T = {}

PATHs = ['/gpfs/data/luilab/CARE_data/', 
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_1of4_082819_310/',
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3aof4_091019_310/',
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3bof4_091119_310/',
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3cof4_091119_310/',
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_4bof4_092319_310/'
        ]

for PATH in PATHs:
    fnames = sorted(os.listdir(PATH))
    print("files in folder:", len(fnames))
    for fname in fnames:
        if 'ImagingDiffusion' not in fname or '.zip' not in fname:
            continue
        T[fname] = os.path.join(PATH, fname)

data1 = pd.DataFrame(pd.Series(T))
data1.rename(columns={0:'save_dir'}, inplace=True)

T_datafolder = T
GUID_datafolder = set([x.split('_')[0] for x in T_datafolder.keys()])
print("data folder zip num: {0}".format(len(T_datafolder)))
print("data folder GUID num: {0}".format(len(GUID_datafolder)))

files in folder: 16755
data folder zip num: 2782
data folder GUID num: 308


In [56]:
# get a dictionary of all DiffusionImaging files we have 
T = {}

PATHs = [#'/gpfs/data/luilab/CARE_data/', 
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_1of4_082819_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3aof4_091019_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3bof4_091119_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3cof4_091119_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_4bof4_092319_310/'
        ]

for PATH in PATHs:
    fnames = sorted(os.listdir(PATH))
    print("files in folder:", len(fnames))
    for fname in fnames:
        if 'ImagingDiffusion' not in fname or '.zip' not in fname:
            continue
        T[fname] = os.path.join(PATH, fname)

data2 = pd.DataFrame(pd.Series(T))
data2.rename(columns={0:'save_dir'}, inplace=True)

T_dti = T
GUID_dtifolder = set([x.split('_')[0] for x in T_dti.keys()])
print("data folder zip num: {0}".format(len(T_dti)))
print("dti folder GUID num: {0}".format(len(GUID_dtifolder)))

files in folder: 2014
files in folder: 403
files in folder: 447
files in folder: 832
files in folder: 636
data folder zip num: 1130
dti folder GUID num: 147


In [46]:
sharedGUID = list(GUID_datafolder & GUID_dtifolder)
print('GUId overlapped: {}'.format(len(sharedGUID)))

GUId overlapped: 147


In [50]:
# print files belonging to the same GUID to find how to link file in folderA to folderB 
guid = sharedGUID[0]
a = [x for x in T_datafolder.keys() if guid in x]
b = [x for x in T_dti.keys() if guid in x]

In [51]:
a

['TBIVN446FFY_ImagingDiffusion_1555341220406_HoQNCFpNmirWzICj7iwMbd58k_8.zip',
 'TBIVN446FFY_ImagingDiffusion_1555341222164_HoQNCFpNmirWzICj7iwMbd58k_9.zip',
 'TBIVN446FFY_ImagingDiffusion_1555341222187_zSb7rysuhEjMFh2SF1cERjpme_8.zip',
 'TBIVN446FFY_ImagingDiffusion_1555341224662_zSb7rysuhEjMFh2SF1cERjpme_9.zip',
 'TBIVN446FFY_ImagingDiffusion_1555341224688_QZe2SvoJygnxT8BFoYj64VUOJ_8.zip',
 'TBIVN446FFY_ImagingDiffusion_1555341227677_QZe2SvoJygnxT8BFoYj64VUOJ_9.zip']

In [52]:
b

['TBIVN446FFY_ImagingDiffusion_1568209951423_HoQNCFpNmirWzICj7iwMbd58k_8.zip',
 'TBIVN446FFY_ImagingDiffusion_1568209952816_HoQNCFpNmirWzICj7iwMbd58k_9.zip',
 'TBIVN446FFY_ImagingDiffusion_1568209953020_zSb7rysuhEjMFh2SF1cERjpme_8.zip',
 'TBIVN446FFY_ImagingDiffusion_1568209954570_zSb7rysuhEjMFh2SF1cERjpme_9.zip',
 'TBIVN446FFY_ImagingDiffusion_1568209954713_QZe2SvoJygnxT8BFoYj64VUOJ_8.zip',
 'TBIVN446FFY_ImagingDiffusion_1568209956399_QZe2SvoJygnxT8BFoYj64VUOJ_9.zip']

data in data folder and updated_dti have not file name overlap

In [121]:
len(np.unique([x.split('_')[0] for x in data2.index.values]))

147

Generate CSV file for DTI folders

In [75]:
# load key in dti folder
T = {}
for key, value in T_dti.items():
    key = key.split('_')
    key.pop(2)
    key = '_'.join(key)
    T[key] = value
data = pd.DataFrame(pd.Series(T))
data.rename(columns={0:'save_dir'}, inplace=True)
data.index.name = 'file_key'

In [102]:
# csv file for DWI scan
name = 'ImagingDiffusion.Image Information.ImgFile'
# csv file for bo
#name = 'ImagingDiffusion.Diffusion Derived Data.ImgFile'
#
ImageDiffusionFile = '/gpfs/data/luilab/FITBIR_META_DATA/assessments/query_result_ImagingDiffusion_2019-07-17T15-35-37668336643334196993.csv'
df_diffusion = pd.read_csv(ImageDiffusionFile)
def map_name(name):
    if isinstance(name, str):
        name = name.split('/')[-1]
        name = name.split('_')
        name.pop(2)
        name = '_'.join(name)
    return name
df_diffusion[name] = df_diffusion[name].apply(map_name)

df_diffusion = df_diffusion.dropna(subset=['ImagingDiffusion.Image Information.ImgFile'])
df_diffusion= df_diffusion.set_index('ImagingDiffusion.Image Information.ImgFile')
df_diffusion.index.name = 'file_key'



In [108]:
merged_data = df_diffusion.merge(data, left_index=True, right_index=True)
print('merged shape: {}'.format(len(merged_data)))

merged shape: 1054


In [118]:
missed_zip_names = data[~data.index.isin(merged_data.index)]

In [122]:
#merged_data.to_csv('./dti_info.csv', index=True)

In [124]:
a = merged_data.drop_duplicates(subset=['ImagingDiffusion.Main.GUID'])

In [127]:
list(zip(*np.unique(a['ImagingDiffusion.Main.CaseContrlInd'].values, return_counts=True)))

[('Case', 58), ('Control', 88)]

In [128]:
list(zip(*np.unique(merged_data['ImagingDiffusion.Main.CaseContrlInd'].values, return_counts=True)))

[('Case', 354), ('Control', 700)]

T1

In [144]:
ImageDiffusionFile = '/gpfs/data/luilab/FITBIR_META_DATA/assessments/query_result_ImagingDiffusion_2019-07-17T15-35-37668336643334196993.csv'
df_diffusion = pd.read_csv(ImageDiffusionFile)
ImageT1File = '/gpfs/data/luilab/FITBIR_META_DATA/assessments/query_result_ImagingMR_2019-07-17T15-31-508831927565068657063.csv'
T1_diffusion = pd.read_csv(ImageT1File,low_memory=False)


In [169]:

def convert_timepoint(x):
    if not isinstance(x, str):
        return 6
    if 'hours' in x:
        return 1
    elif 'sympt' in x:
        return 2
    elif 'days'in x:
        return 3
    elif 'months' in x:
        return 4
    return 0

diffusion_keys = []
for i, value in df_diffusion.iterrows():
    guid = str(value['ImagingDiffusion.Main.GUID']) 
    subjectid = str(int(value['ImagingDiffusion.Main.SubjectIDNum'])) if value['ImagingDiffusion.Main.SubjectIDNum'] > 0 else 'NA'
    timepoint = str(convert_timepoint(value['ImagingDiffusion.Main.GeneralNotesTxt']))
    
    key = '-'.join([guid, subjectid, timepoint])
    diffusion_keys.append(key)
    
diffusion_keys = [x for x in diffusion_keys if 'nan' not in x]
    

In [171]:
ti_keys = []
for i, value in T1_diffusion.iterrows():
    guid = str(value['ImagingMR.Main.GUID']) 
    subjectid = str(int(value['ImagingMR.Main.SubjectIDNum'])) if value['ImagingMR.Main.SubjectIDNum'] > 0 else 'NA'
    timepoint = str(convert_timepoint(value['ImagingMR.Main.GeneralNotesTxt']))
    
    key = '-'.join([guid, subjectid, timepoint])
    ti_keys.append(key)
    
ti_keys = [x for x in ti_keys if 'nan' not in x]


In [177]:
len(set(diffusion_keys))

1072

In [178]:
len(set(ti_keys))

1075

In [180]:
len(set(diffusion_keys) & set(ti_keys))

1072

Process Data

In [100]:
# get a dictionary of all DiffusionImaging files we have 
T = {}

PATHs = ['/gpfs/data/luilab/CARE_data/', 
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_1of4_082819_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3aof4_091019_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3bof4_091119_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3cof4_091119_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_4bof4_092319_310/'
        ]

for PATH in PATHs:
    fnames = sorted(os.listdir(PATH))
    print(len(fnames))
    for fname in fnames:
        if 'ImagingDiffusion' not in fname or '.zip' not in fname:
            continue
        T[fname] = os.path.join(PATH, fname)

data = pd.DataFrame(pd.Series(T))
data.rename(columns={0:'save_dir'}, inplace=True)

data.shape

16755
2014
403
447
832
636


(3912, 1)

In [126]:
# read ImageDiffusion file 
ImageDiffusionFile = '/gpfs/data/luilab/FITBIR_META_DATA/assessments/query_result_ImagingDiffusion_2019-07-17T15-35-37668336643334196993.csv'
df_diffusion = pd.read_csv(ImageDiffusionFile)
df_diffusion['FileName'] = df_diffusion['ImagingDiffusion.Image Information.ImgFile'].map(lambda x: x.split('/')[-1] if isinstance(x, str) else x)

merged_df = df_diffusion.merge(data, left_on='FileName', right_index=True)
merged_df.shape

print("{}/{} of zip is in our drive:".format(merged_df.shape[0], df_diffusion.dropna(subset=['Dataset']).shape[0]))

unique_GUIDs = merged_df['ImagingDiffusion.Main.GUID'].unique()
print(f"number of unqiue GUID we have: {len(unique_GUIDs)}")

UNZIP

In [128]:
SAVE_PATH = '../Diffusion_Imaging/'
for i, row in merged_df.iterrows():
    print(i)
    file_name = row['FileName'][:-4]
    file_name_dir = os.path.join(SAVE_PATH, file_name)
    if not os.path.isdir(file_name_dir):
        os.mkdir(file_name_dir)
    source_zip = row['save_dir']
    os.system('unzip {} -d {}'.format(source_zip, \
                                  file_name_dir))

0
1
2
3
4
5
6
10
14
15
19
20
21
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285

2501
2502
2503
2504
2505
2506
2507
2508
2512
2513
2514
2515
2516
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550


Phase Encoding

In [9]:
from util import find_nth
def find_PE(path):
    found = False
    PE_folder_name = list(os.walk(path))[2][1][0]
    #print(PE_folder_name)
    start = max(find_nth(PE_folder_name, 'b0', 1), \
                find_nth(PE_folder_name, 'b1000', 1),\
                find_nth(PE_folder_name, 'b2000', 1))
    start = PE_folder_name.find("_", start)
    PE = PE_folder_name[start+1:start+3]    
    if PE not in ['AP', 'PA']:
        print(PE_folder_name)
    return PE

def if_contains_nii(path):
    for i, level in enumerate(os.walk(path)):
        if any('.nii' in x for x in level[2]):
            return 1
    return 0

In [10]:
dir1 = '../Diffusion_Imaging/'
PEs = []
for fname in os.listdir('../Diffusion_Imaging/'):
    path = os.path.join(dir1, fname)
    PE = find_PE(path)
    PEs.append(PE)
    






11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
12-SB_DTI_MCW
12-SB_DTI_MCW
11-SB_DTI_MCW
12-SB_DTI_MCW
12-SB_DTI_MCW
11-SB_DTI_MCW
13-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
12-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
12-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
12-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
15-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
11-SB_DTI_MCW
12-SB_DTI_MCW
11-SB_

In [19]:
list(zip(*np.unique(PEs, return_counts=True)))

[('11', 262),
 ('12', 28),
 ('13', 6),
 ('14', 1),
 ('15', 2),
 ('16', 1),
 ('AP', 1010),
 ('PA', 548)]

Check Overlap Between DTI and TI

In [24]:
ImageDiffusionFile = '/gpfs/data/luilab/FITBIR_META_DATA/assessments/query_result_ImagingDiffusion_2019-07-17T15-35-37668336643334196993.csv'
df_diffusion = pd.read_csv(ImageDiffusionFile)
print(df_diffusion.shape)


(2551, 155)


In [25]:
TI_file = '/gpfs/data/luilab/FITBIR_META_DATA/assessments/query_result_ImagingMR_2019-07-17T15-31-508831927565068657063.csv'
TI_csv = pd.read_csv(TI_file, low_memory=False)
print(TI_csv.shape)



(5827, 134)


In [26]:
merged_csv = TI_csv.merge(df_diffusion, left_on='ImagingMR.Main.GUID', right_on='ImagingDiffusion.Main.GUID',how='inner')

In [27]:
merged_csv.shape

(39517, 289)

In [28]:
df_diffusion.shape

(2551, 155)

In [19]:
df_diffusion

,Study ID,Dataset,ImagingDiffusion.Main.GUID,ImagingDiffusion.Main.SubjectIDNum,ImagingDiffusion.Main.AgeYrs,ImagingDiffusion.Main.VisitDate,ImagingDiffusion.Main.SiteName,ImagingDiffusion.Main.DaysSinceBaseline,ImagingDiffusion.Main.CaseContrlInd,ImagingDiffusion.Main.GeneralNotesTxt,...,ImagingDiffusion.Diffusion Direction Data.ImgSptllyRegistrTmpltInd,ImagingDiffusion.Diffusion Direction Data.ImgT2WSpinEchoAcquireInd,ImagingDiffusion.Diffusion Direction Data.ImgDiffusionBValFile,ImagingDiffusion.Diffusion Direction Data.ImgDiffusionBVecFile,ImagingDiffusion.Diffusion Derived Data.ImgDiffusionDerivedDataTyp,ImagingDiffusion.Diffusion Derived Data.ImgDiffusionDerivedDataTypOTH,ImagingDiffusion.Diffusion Derived Data.ImgFile,ImagingDiffusion.Diffusion Measurements.ImgDiffusionMeasrTyp,ImagingDiffusion.Diffusion Measurements.ImgDiffusionMeasrOTH,ImagingDiffusion.Diffusion Measurements.ImgDiffusionReslt
0,310.0,FITBIR-DATA0006022,TBIBA140TWP,2470.0,20.0,2017-04-19T00:00:00Z,NaN,NaN,Case,7 days Post-Unrestricted Return to Play,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,310.0,FITBIR-DATA0006022,TBIEX081KDF,1391.0,20.0,2016-08-17T00:00:00Z,NaN,NaN,Case,Asymptomatic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,310.0,FITBIR-DATA0006022,TBIDL353RYK,1759.0,22.0,2017-03-23T00:00:00Z,NaN,NaN,Case,6 months post-injury,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,310.0,FITBIR-DATA0006022,TBIEX081KDF,1390.0,20.0,2017-02-06T00:00:00Z,NaN,NaN,Case,6 months post-injury,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,310.0,FITBIR-DATA0006022,TBIDL353RYK,1759.0,21.0,2016-10-27T00:00:00Z,NaN,NaN,Case,24-48 hours,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,310.0,FITBIR-DATA0006022,TBIBA140TWP,2470.0,20.0,2017-10-04T00:00:00Z,NaN,NaN,Case,6 months post-injury,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,310.0,FITBIR-DATA0006022,TBIEX081KDF,420.0,19.0,2015-08-13T00:00:00Z,NaN,NaN,Case,Asymptomatic,...,NaN,NaN,NaN,NaN,B0 field map,NaN,/imagingdiffusion_output_1555339956653/TBIEX08...,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,B0 field map,NaN,/imagingdiffusion_output_1555339956653/TBIEX08...,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,B0 field map,NaN,/imagingdiffusion_output_1555339956653/TBIEX08...,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,B0 field map,NaN,/imagingdiffusion_output_1555339956653/TBIEX08...,NaN,NaN,NaN


In [29]:
A = df_diffusion.dropna(subset=['Dataset'])
A = A['ImagingDiffusion.Main.GUID'] + '-' + A['ImagingDiffusion.Main.GeneralNotesTxt']
A = pd.DataFrame(A).rename(columns={0:'name-time'})

In [30]:
B = TI_csv.dropna(subset=['Dataset'])
B = B['ImagingMR.Main.GUID'] + '-' + B['ImagingMR.Main.GeneralNotesTxt']
B = pd.DataFrame(B).rename(columns={0:'name-time'})



In [31]:
A.merge(B, left_on='name-time', right_on='name-time')


,name-time
0,TBIBA140TWP-7 days Post-Unrestricted Return to...
1,TBIBA140TWP-7 days Post-Unrestricted Return to...
2,TBIBA140TWP-7 days Post-Unrestricted Return to...
3,TBIBA140TWP-7 days Post-Unrestricted Return to...
4,TBIBA140TWP-7 days Post-Unrestricted Return to...
5,TBIBA140TWP-7 days Post-Unrestricted Return to...
6,TBIBA140TWP-7 days Post-Unrestricted Return to...
7,TBIBA140TWP-7 days Post-Unrestricted Return to...
8,TBIBA140TWP-7 days Post-Unrestricted Return to...
9,TBIBA140TWP-7 days Post-Unrestricted Return to...


In [45]:
B.drop_duplicates(subset=['name-time'])

,name-time
0,TBIBT336EPU-6 months post-injury
1,TBICR008BFT-24-48 hours
2,TBIYJ357BMK-24-48 hours
3,TBIZZ437LYN-Asymptomatic
4,TBIEN947FJE-7 days Post-Unrestricted Return to...
5,TBIJY283VKB-24-48 hours
6,TBIBT578WHF-24-48 hours
7,TBI_INVFU773VD1-6 months post-injury
8,TBIDA795WGK-Asymptomatic
9,TBILP098UN0-24-48 hours


In [32]:
B.sort_values(by='name-time')

,name-time
1837,TBIAE634EUZ-24-48 hours
2527,TBIAE634EUZ-24-48 hours
1444,TBIAE634EUZ-24-48 hours
2365,TBIAE634EUZ-24-48 hours
5517,TBIAE634EUZ-24-48 hours
1557,TBIAE634EUZ-6 months post-injury
5383,TBIAE634EUZ-6 months post-injury
1677,TBIAE634EUZ-6 months post-injury
2760,TBIAE634EUZ-6 months post-injury
2289,TBIAE634EUZ-6 months post-injury


In [33]:
A

,name-time
0,TBIBA140TWP-7 days Post-Unrestricted Return to...
1,TBIEX081KDF-Asymptomatic
2,TBIDL353RYK-6 months post-injury
3,TBIEX081KDF-6 months post-injury
4,TBIDL353RYK-24-48 hours
5,TBIBA140TWP-6 months post-injury
6,TBIEX081KDF-Asymptomatic
10,TBIDB663GHZ-24-48 hours
14,TBIDL353RYK-Asymptomatic
15,TBIKW626MME-7 days Post-Unrestricted Return to...


In [6]:
file = os.path.join(PATH, 'TBIAE634EUZ_ImagingDiffusion_1555099100503_G9LFVHGNnqf0WlX9bokCPU9CF_11.zip')
os.system('unzip {} -d {}'.format(file, \
                                  './'))


0

In [42]:
DTI_Final_PATHs = ['/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_1of4_082819_310/',
                   '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3aof4_091019_310/',
                   '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3bof4_091119_310/',
                   '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3cof4_091119_310/',
                   '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_4bof4_092319_310/']

for DTI_Final_PATH in DTI_Final_PATHs:
    files = sorted(os.listdir(DTI_Final_PATH))

['ImagingDiffusion_pv_mapping.csv',
 'TBIAK865LX3_ImagingDiffusion_1569269555894.jpg',
 'TBIAK865LX3_ImagingDiffusion_1569269555894_U2klmX2hqLRi4DyEzCGmMutyS_8.zip',
 'TBIAK865LX3_ImagingDiffusion_1569269555894_bvals.txt',
 'TBIAK865LX3_ImagingDiffusion_1569269555894_bvecs.txt',
 'TBIAK865LX3_ImagingDiffusion_1569269556494.jpg',
 'TBIAK865LX3_ImagingDiffusion_1569269556494_U2klmX2hqLRi4DyEzCGmMutyS_9.zip',
 'TBIAK865LX3_ImagingDiffusion_1569269556494_bvals.txt',
 'TBIAK865LX3_ImagingDiffusion_1569269556494_bvecs.txt',
 'TBIAK865LX3_ImagingDiffusion_1569269556563.jpg',
 'TBIAK865LX3_ImagingDiffusion_1569269556563_bvals.txt',
 'TBIAK865LX3_ImagingDiffusion_1569269556563_bvecs.txt',
 'TBIAK865LX3_ImagingDiffusion_1569269556563_ykuLRy8Lgc805zEtMb1rr0ron_8.zip',
 'TBIAK865LX3_ImagingDiffusion_1569269557165.jpg',
 'TBIAK865LX3_ImagingDiffusion_1569269557165_bvals.txt',
 'TBIAK865LX3_ImagingDiffusion_1569269557165_bvecs.txt',
 'TBIAK865LX3_ImagingDiffusion_1569269557165_ykuLRy8Lgc805zEtMb1rr0

In [113]:
PATH = '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3aof4_091019_310/'
files_3aof4 = [os.path.join(PATH, x) for x in os.listdir(PATH)]
files_3aof4 = sorted(files_3aof4)
print(len(files_3aof4))

403


In [116]:
PATH = '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3bof4_091119_310/'
files_3bof4 = [os.path.join(PATH, x) for x in os.listdir(PATH)]
files_3bof4 = sorted(files_3bof4)
print(len(files_3bof4))

447


In [117]:
PATH = '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3cof4_091119_310/'
files_3cof4 = [os.path.join(PATH, x) for x in os.listdir(PATH)]
files_3cof4 = sorted(files_3bof4)
print(len(files_3bof4))

832


In [101]:
names = [x.split('/')[-1] for x in files]
names = [x[:x.find('_')] for x in names]

In [102]:
GUIDs = set(names)

In [103]:
len(GUIDs)

305

In [105]:
GUIDs

{'.TBIZN166LJY',
 'Simoa N4PA Advantage Kit, Kit Instructions for HD-1 030618.pd',
 'TBIAE634EUZ',
 'TBIAG941ZAD',
 'TBIAJ330AC2',
 'TBIAK358WYF',
 'TBIAK838YR3',
 'TBIAK865LX3',
 'TBIAM804GRN',
 'TBIAV824LMK',
 'TBIAX166PYB',
 'TBIAZ817EYK',
 'TBIBA122HW3',
 'TBIBA140TWP',
 'TBIBA423BM6',
 'TBIBA764HRH',
 'TBIBD533AKK',
 'TBIBD941WKU',
 'TBIBE389TLQ',
 'TBIBG331LZX',
 'TBIBJ248BHJ',
 'TBIBL649MKK',
 'TBIBL749VD6',
 'TBIBP604ERQ',
 'TBIBT336EPU',
 'TBIBT350AWY',
 'TBIBT578WHF',
 'TBIBU887ZBJ',
 'TBIBX189XU1',
 'TBIBY712FUD',
 'TBIBZ967EE4',
 'TBICB604EKU',
 'TBICC177VED',
 'TBICG850WEN',
 'TBICJ342FKR',
 'TBICJ936JMA',
 'TBICK676PXX',
 'TBICL212TYK',
 'TBICM229RHC',
 'TBICR008BFT',
 'TBICR864CMD',
 'TBICT108MUU',
 'TBICU696PFJ',
 'TBICV764APZ',
 'TBICW412YJX',
 'TBICX014CFL',
 'TBICZ031GEX',
 'TBICZ061LW4',
 'TBIDA642TJK',
 'TBIDA795WGK',
 'TBIDB663GHZ',
 'TBIDC043EBA',
 'TBIDD569TVE',
 'TBIDJ805FVM',
 'TBIDJ815AEF',
 'TBIDL110TM0',
 'TBIDL353RYK',
 'TBIDN578PYP',
 'TBIDR332KTD',
 'TBI

In [88]:
files

['/gpfs/data/luilab/CARE_data/.TBIXM072VWP_ImagingDiffusion_1555353314278_5PrmljkOA9Pe9dmp6YSqf9f7a_8.zip.Id0aAX',
 '/gpfs/data/luilab/CARE_data/.TBIXM072VWP_ImagingDiffusion_1555353314278_5PrmljkOA9Pe9dmp6YSqf9f7a_8.zip.nd1blz',
 '/gpfs/data/luilab/CARE_data/Simoa N4PA Advantage Kit, Kit Instructions for HD-1 030618.pdf',
 '/gpfs/data/luilab/CARE_data/TBIAE634EUZ_ImagingDiffusion_1555099100503.jpg',
 '/gpfs/data/luilab/CARE_data/TBIAE634EUZ_ImagingDiffusion_1555099100503_G9LFVHGNnqf0WlX9bokCPU9CF_11.zip',
 '/gpfs/data/luilab/CARE_data/TBIAE634EUZ_ImagingDiffusion_1555099100670.jpg',
 '/gpfs/data/luilab/CARE_data/TBIAE634EUZ_ImagingDiffusion_1555099100670_Xvobnk6DuIgaz6tvop3C57R92_11.zip',
 '/gpfs/data/luilab/CARE_data/TBIAE634EUZ_ImagingDiffusion_1555099101143.jpg',
 '/gpfs/data/luilab/CARE_data/TBIAE634EUZ_ImagingDiffusion_1555099101143_Mq9rsXBDRJfF5Ih5q0BHUZith_11.zip',
 '/gpfs/data/luilab/CARE_data/TBIAE634EUZ_ImagingDiffusion_1555099101308.jpg',
 '/gpfs/data/luilab/CARE_data/TBIAE

In [43]:
T = {}

PATHs = [#'/gpfs/data/luilab/CARE_data/', 
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_1of4_082819_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3aof4_091019_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3bof4_091119_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3cof4_091119_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_4bof4_092319_310/'
        ]
names = []
for PATH in PATHs:
    fnames = sorted(os.listdir(PATH))
    names += [os.path.join(PATH, fname) for fname in fnames if '_bval' in fname]
    

In [12]:
lens = []
for fname in names:
    file = open(fname, "rt")
    data = file.read()
    words = data.split()
    lens.append(len(words))
    

In [15]:
from collections import defaultdict

In [16]:
t = defaultdict(list)

In [17]:
for i in range(len(lens)):
    t[lens[i]].append(names[i])

In [29]:
list(zip(*np.unique(lens, return_counts=True)))

[(12, 1), (50, 2), (61, 380), (64, 119), (65, 11), (68, 538), (69, 7)]

In [30]:
{key : len(t[key])  for key in t}

{64: 119, 61: 380, 12: 1, 68: 538, 65: 11, 50: 2, 69: 7}

###### len(os.listdir('/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3bof4_091119_310/'))

In [6]:
PATHs = ['/gpfs/data/luilab/CARE_data/', 
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_1of4_082819_310/',
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3aof4_091019_310/',
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3bof4_091119_310/',
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3cof4_091119_310/',
         #'/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_4bof4_092319_310/'
        ]
PATH = '/gpfs/data/luilab/CARE_data/'
fnames = sorted(os.listdir(PATH))

In [8]:
fnames = [fname for fname in fnames if "ImagingMR" in fname and "zip" in fname]

In [10]:
fnames = [fname for fname in fnames if "TBIAE634EUZ" in fname]

In [11]:
fnames

['TBIAE634EUZ_ImagingMR_1553261224494_G9LFVHGNnqf0WlX9bokCPU9CF_6.zip',
 'TBIAE634EUZ_ImagingMR_1553261224538_Xvobnk6DuIgaz6tvop3C57R92_6.zip',
 'TBIAE634EUZ_ImagingMR_1553261224580_Mq9rsXBDRJfF5Ih5q0BHUZith_6.zip',
 'TBIAE634EUZ_ImagingMR_1553261224621_qMhKgvAgQrkYEZJQfEK2QMlFb_6.zip',
 'TBIAE634EUZ_ImagingMR_1553545288375_G9LFVHGNnqf0WlX9bokCPU9CF_12.zip',
 'TBIAE634EUZ_ImagingMR_1553545288922_Xvobnk6DuIgaz6tvop3C57R92_12.zip',
 'TBIAE634EUZ_ImagingMR_1553545289875_Mq9rsXBDRJfF5Ih5q0BHUZith_12.zip',
 'TBIAE634EUZ_ImagingMR_1553545290313_qMhKgvAgQrkYEZJQfEK2QMlFb_12.zip',
 'TBIAE634EUZ_ImagingMR_1553864682754_G9LFVHGNnqf0WlX9bokCPU9CF_4.zip',
 'TBIAE634EUZ_ImagingMR_1553864683023_Xvobnk6DuIgaz6tvop3C57R92_4.zip',
 'TBIAE634EUZ_ImagingMR_1553864683648_Mq9rsXBDRJfF5Ih5q0BHUZith_4.zip',
 'TBIAE634EUZ_ImagingMR_1553864683918_qMhKgvAgQrkYEZJQfEK2QMlFb_4.zip',
 'TBIAE634EUZ_ImagingMR_1553870670822_G9LFVHGNnqf0WlX9bokCPU9CF_5.zip',
 'TBIAE634EUZ_ImagingMR_1553870671753_Xvobnk6DuIgaz6tvop3C57

In [15]:
TI_file = '/gpfs/data/luilab/FITBIR_META_DATA/assessments/query_result_ImagingMR_2019-07-17T15-31-508831927565068657063.csv'
TI_csv = pd.read_csv(TI_file, low_memory=False)
print(TI_csv.shape)
ImageDiffusionFile = '/gpfs/data/luilab/FITBIR_META_DATA/assessments/query_result_ImagingDiffusion_2019-07-17T15-35-37668336643334196993.csv'
df_diffusion = pd.read_csv(ImageDiffusionFile)
print(df_diffusion.shape)



(5827, 134)
(2551, 155)


In [30]:
candidate_t1 = TI_csv[TI_csv['ImagingMR.Main.GUID'] == "TBIZN166LJY"]

In [31]:
candidate_dti = df_diffusion[df_diffusion['ImagingDiffusion.Main.GUID'] == "TBIZN166LJY"]

In [32]:
candidate_dti = candidate_dti[candidate_dti['ImagingDiffusion.Image Information.ImgFile'].str.contains("FwI5HfqsmyMup5jdJTqFy5iJK")]

In [33]:
candidate_dti['ImagingDiffusion.Image Information.ImgFile'].values

array(['/imagingdiffusion_output_1555361690443/TBIZN166LJY_ImagingDiffusion_1555361715127_FwI5HfqsmyMup5jdJTqFy5iJK_9.zip',
       '/imagingdiffusion_output_1555361690443/TBIZN166LJY_ImagingDiffusion_1555361714277_FwI5HfqsmyMup5jdJTqFy5iJK_8.zip'],
      dtype=object)

In [34]:
candidate_dti

,Study ID,Dataset,ImagingDiffusion.Main.GUID,ImagingDiffusion.Main.SubjectIDNum,ImagingDiffusion.Main.AgeYrs,ImagingDiffusion.Main.VisitDate,ImagingDiffusion.Main.SiteName,ImagingDiffusion.Main.DaysSinceBaseline,ImagingDiffusion.Main.CaseContrlInd,ImagingDiffusion.Main.GeneralNotesTxt,...,ImagingDiffusion.Diffusion Direction Data.ImgSptllyRegistrTmpltInd,ImagingDiffusion.Diffusion Direction Data.ImgT2WSpinEchoAcquireInd,ImagingDiffusion.Diffusion Direction Data.ImgDiffusionBValFile,ImagingDiffusion.Diffusion Direction Data.ImgDiffusionBVecFile,ImagingDiffusion.Diffusion Derived Data.ImgDiffusionDerivedDataTyp,ImagingDiffusion.Diffusion Derived Data.ImgDiffusionDerivedDataTypOTH,ImagingDiffusion.Diffusion Derived Data.ImgFile,ImagingDiffusion.Diffusion Measurements.ImgDiffusionMeasrTyp,ImagingDiffusion.Diffusion Measurements.ImgDiffusionMeasrOTH,ImagingDiffusion.Diffusion Measurements.ImgDiffusionReslt
2184,310.0,FITBIR-DATA0006025,TBIZN166LJY,3788.0,19.0,2018-02-28T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2419,310.0,FITBIR-DATA0006025,TBIZN166LJY,3788.0,19.0,2018-02-28T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
PATHs = [#'/gpfs/data/luilab/CARE_data/', 
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_1of4_082819_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3aof4_091019_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3bof4_091119_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_3cof4_091119_310/',
         '/gpfs/data/luilab/FITBIR_META_DATA/updated_data/DTI_Final_4bof4_092319_310/'
        ]
names = []
for PATH in PATHs:
    names += [os.path.join(PATH,name) for name in os.listdir(PATH)]

In [50]:
names = [name for name in names if "MR9JuZKbUmo4h50eLpmAMi7ih" in name]

In [52]:
candidate_dti

,Study ID,Dataset,ImagingDiffusion.Main.GUID,ImagingDiffusion.Main.SubjectIDNum,ImagingDiffusion.Main.AgeYrs,ImagingDiffusion.Main.VisitDate,ImagingDiffusion.Main.SiteName,ImagingDiffusion.Main.DaysSinceBaseline,ImagingDiffusion.Main.CaseContrlInd,ImagingDiffusion.Main.GeneralNotesTxt,...,ImagingDiffusion.Diffusion Direction Data.ImgSptllyRegistrTmpltInd,ImagingDiffusion.Diffusion Direction Data.ImgT2WSpinEchoAcquireInd,ImagingDiffusion.Diffusion Direction Data.ImgDiffusionBValFile,ImagingDiffusion.Diffusion Direction Data.ImgDiffusionBVecFile,ImagingDiffusion.Diffusion Derived Data.ImgDiffusionDerivedDataTyp,ImagingDiffusion.Diffusion Derived Data.ImgDiffusionDerivedDataTypOTH,ImagingDiffusion.Diffusion Derived Data.ImgFile,ImagingDiffusion.Diffusion Measurements.ImgDiffusionMeasrTyp,ImagingDiffusion.Diffusion Measurements.ImgDiffusionMeasrOTH,ImagingDiffusion.Diffusion Measurements.ImgDiffusionReslt
1625,310.0,FITBIR-DATA0006023,TBIWK466PHH,1705.0,20.0,2016-10-19T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2122,310.0,FITBIR-DATA0006023,TBIWK466PHH,1705.0,20.0,2016-10-19T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
candidate_t1 = TI_csv[TI_csv['ImagingMR.Main.GUID'] == "TBIZN166LJY"]
candidate_t1 = candidate_t1[candidate_t1['ImagingMR.Main.GeneralNotesTxt'].str.contains("24-48 hours")]

In [36]:
candidate_t1.to_csv('./temp2.csv')

In [37]:
candidate_t1

,Study ID,Dataset,ImagingMR.Main.GUID,ImagingMR.Main.SubjectIDNum,ImagingMR.Main.AgeYrs,ImagingMR.Main.VisitDate,ImagingMR.Main.SiteName,ImagingMR.Main.DaysSinceBaseline,ImagingMR.Main.CaseContrlInd,ImagingMR.Main.GeneralNotesTxt,...,ImagingMR.Magnetic Resonance Information.ImgBaseResolutionVal,ImagingMR.Magnetic Resonance Information.ImgShotCt,ImagingMR.Magnetic Resonance Information.ImgSignalAvgNum,ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilName,ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilChannelCt,ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilChannelOTH,ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilFunctTyp,ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilTyp,ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilOTH,ImagingMR.Magnetic Resonance RF Coil.ImgRFCoilCat
200,310,FITBIR-DATA0005980,TBIZN166LJY,3788,19,2018-02-28T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,1.0,NaN,32,NaN,NaN,NaN,NaN,NaN
335,310,FITBIR-DATA0005980,TBIZN166LJY,3788,19,2018-02-28T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,1.0,NaN,32,NaN,NaN,NaN,NaN,NaN
1242,310,FITBIR-DATA0005980,TBIZN166LJY,3788,19,2018-02-28T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,1.0,NaN,32,NaN,NaN,NaN,NaN,NaN
3396,310,FITBIR-DATA0006001,TBIZN166LJY,3788,19,2018-02-28T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,1.0,NaN,32,NaN,NaN,NaN,NaN,NaN
4458,310,FITBIR-DATA0006001,TBIZN166LJY,3788,19,2018-02-28T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,1.0,NaN,32,NaN,NaN,NaN,NaN,NaN
4907,310,FITBIR-DATA0006014,TBIZN166LJY,3788,19,2018-02-28T00:00:00Z,NaN,NaN,Control,24-48 hours,...,NaN,NaN,1.0,NaN,32,NaN,NaN,NaN,NaN,NaN
